## Дополнительные подсистемы


## Rate Limiting (Ограничение нагрузки)
Зачем? Защита от DDoS, Brute-force, честное распределение ресурсов.

### Алгоритмы Rate Limiting
1.  **Token Bucket (Ведро с токенами)**:
    *   В ведро капают токены с постоянной скоростью.
    *   Запрос забирает токен.
    *   Нет токена -> 429 Too Many Requests.
    *   *Плюс*: Позволяет кратковременные всплески (Bursts).
2.  **Leaky Bucket (Протекающее ведро)**:
    *   Запросы попадают в очередь (ведро).
    *   Обрабатываются с фиксированной скоростью (как вода из дырки).
    *   *Плюс*: Сглаживает нагрузку (Traffic Shaping).
3.  **Fixed Window Counter**: Сбрасываем счетчик каждую минуту.
    *   *Минус*: Проблема на стыке окон (можно послать 2x лимита).
4.  **Sliding Window Log**: Храним таймстемпы каждого запроса. (Точно, но дорого по памяти).

### Где размещать?
*   **API Gateway / Reverse Proxy**: Лучшее место (nginx, kong).
*   **Sidecar**: В микросервисной архитектуре.

---

## Мониторинг и Логирование
"Если вы это не мониторите, этого не существует".

### Типы телеметрии:
1.  **Метрики (Metrics)**: Числа (CPU, RPS, Latency). Дешево хранить.
    *   Инструменты: **Prometheus + Grafana**, Zabbix.
2.  **Логи (Logs)**: События (Error stacktrace). Дорого хранить.
    *   Инструменты: **ELK Stack (Elasticsearch, Logstash, Kibana)**, Loki.
3.  **Трейсинг (Tracing)**: Путь запроса через микросервисы.
    *   Инструменты: **Jaeger**, Zipkin.

### Паттерны сбора:
*   **Push (Graphite)**: Сервис сам шлет метрики.
*   **Pull (Prometheus)**: Система мониторинга ходит по сервисам и собирает метрики (/metrics).


## Практика: Rate Limiter (Token Bucket)


### Алгоритм "Ведро с токенами"
*   Есть ведро емкостью `capacity`.
*   В него капают токены со скоростью `refill_rate`.
*   Каждый запрос забирает 1 токен.
*   Если токенов нет -> отказ (429).

Этот алгоритм позволяет сглаживать нагрузку, но пропускает кратковременные всплески (Bursts) до размера ведра.


In [ ]:

import time

class TokenBucket:
    def __init__(self, capacity, refill_rate):
        self.capacity = capacity          # Макс токенов (размер ведра)
        self.refill_rate = refill_rate    # Токенов в секунду
        self.tokens = capacity            # Текущее кол-во токенов
        self.last_refill = time.time()    # Время последнего пополнения

    def _refill(self):
        now = time.time()
        delta = now - self.last_refill
        # Сколько накапало за прошедшее время
        tokens_to_add = delta * self.refill_rate
        
        self.tokens = min(self.capacity, self.tokens + tokens_to_add)
        self.last_refill = now

    def allow_request(self, tokens_needed=1):
        self._refill()
        
        if self.tokens >= tokens_needed:
            self.tokens -= tokens_needed
            return True
        else:
            return False

# --- Тест ---
# Емкость 5, скорость 1 токен/сек
limiter = TokenBucket(capacity=5, refill_rate=1)

print("--- Старт (ведро полное: 5) ---")
for i in range(1, 8):
    allowed = limiter.allow_request()
    status = "✅ OK" if allowed else "⛔ 429 Too Many Requests"
    print(f"Запрос {i}: {status} (Токенов осталось: {int(limiter.tokens)})")
    
    if i == 5:
        print("--- Ждем 2 секунды (должно накапать ~2 токена) ---")
        time.sleep(2)
